In [3]:
import os

from dotenv import dotenv_values
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

env_vars = dotenv_values('.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    chat.predict('What is the recipe for soju?')
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [15.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Soju is a traditional Korean distilled alcoholic beverage. The recipe for soju involves a few simple ingredients and a fermentation process. Here's a basic recipe:\n\nIngredients:\n- 1 kg of rice (short-grain rice is commonly used)\n- 1 kg of nuruk (a Korean fermentation starter)\n- 10 liters of water\n- 400-500 grams of sugar (optional, for sweetness)\n- Distilled water (for dilution)\n\nInstructions:\n1. Rinse the rice thoroughly until the water runs clear. Soak the rice in water for about 1-2 hours.\n2. Drain the soaked rice and steam it until fully cooked.\n3. Allow the cooked rice to cool down to room temperature.\n4. In a large container or pot, mix the cooled rice, nuruk, and water. Stir well to combine.\n5. Cover the container with a clea